In [302]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import altair as alt

# Exercise1 - load data

In [303]:
url = "https://github.com/nickeubank/MIDS_Data/raw/master/smoking.csv"
init = pd.read_csv(url)
smoking = init.rename({"bwt.oz":"bwt"}, axis='columns')
smoking.head(10)

,id,date,gestation,bwt,parity,mrace,mage,med,mht,mpregwt,inc,smoke
0,4604,1598,148,116,7,7,28,1,66,135,2,0
1,7435,1527,181,110,7,7,27,1,64,133,1,0
2,7722,1563,204,55,11,7,35,3,65,140,6,0
3,2026,1503,225,132,4,7,28,2,67,148,3,0
4,3553,1638,233,105,4,7,34,3,61,130,3,0
5,3491,1705,234,85,7,7,33,1,67,130,2,0
6,6757,1444,234,97,0,6,26,5,65,112,6,0
7,6153,1405,235,129,3,7,24,4,66,135,1,0
8,8187,1669,236,63,0,5,24,5,58,99,7,0
9,8403,1669,241,128,0,7,17,1,64,126,2,0


# Exercise 2 -plot

In [304]:
smoking1 = smoking.copy().loc[smoking['gestation']>225]

In [305]:
smoking1.head()

,id,date,gestation,bwt,parity,mrace,mage,med,mht,mpregwt,inc,smoke
4,3553,1638,233,105,4,7,34,3,61,130,3,0
5,3491,1705,234,85,7,7,33,1,67,130,2,0
6,6757,1444,234,97,0,6,26,5,65,112,6,0
7,6153,1405,235,129,3,7,24,4,66,135,1,0
8,8187,1669,236,63,0,5,24,5,58,99,7,0


In [306]:
smoke_plot = (
    alt.Chart(smoking1)
    .mark_point()
    .encode(
        x=alt.X("gestation", scale=alt.Scale(zero=False)),
        y=alt.Y("bwt", scale=alt.Scale(zero=False)),
        color='smoke:N',
    )
)

smoke_plot + smoke_plot.transform_regression(
    "gestation", "bwt", groupby=['smoke']
).mark_line()

/Users/super_enbicycle/opt/miniconda3/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [307]:
ex2_birth_weight_lower = "yes"

# Exercise 3

In [308]:
model1 = smf.ols('bwt ~ gestation + smoke', data=smoking1).fit()
model1.summary()
ex3_smoking_coefficient = model1.params['smoke']
ex3_smoking_coefficient


-7.878421244482617

# Exercise 4

In [309]:
model2 = smf.ols('bwt ~ gestation + smoke + mpregwt+ C(mrace)', data=smoking1).fit()
model2.summary()
ex4_smoking_coefficient = model2.params['smoke']
ex4_smoking_coefficient

-7.895835256029642

# Exericise 5

In [310]:
#smoking["white"] = np.where(smoking["mrace"]<=5, "True", "False") 
smoking1['white'] = smoking1['mrace'] < 6
smoking1.head(10)

,id,date,gestation,bwt,parity,mrace,mage,med,mht,mpregwt,inc,smoke,white
4,3553,1638,233,105,4,7,34,3,61,130,3,0,False
5,3491,1705,234,85,7,7,33,1,67,130,2,0,False
6,6757,1444,234,97,0,6,26,5,65,112,6,0,False
7,6153,1405,235,129,3,7,24,4,66,135,1,0,False
8,8187,1669,236,63,0,5,24,5,58,99,7,0,True
9,8403,1669,241,128,0,7,17,1,64,126,2,0,False
10,2960,1476,244,107,1,5,20,2,58,97,1,0,True
11,129,1562,245,132,2,7,23,1,65,140,2,0,False
12,8172,1691,246,130,1,0,19,2,62,118,1,0,True
13,1119,1548,246,97,6,7,37,2,63,150,2,0,False


In [311]:
model3 = smf.ols('bwt ~ gestation + smoke + mpregwt + white + smoke:white', data=smoking1).fit()
model3.params
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    bwt   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     54.81
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           1.91e-49
Time:                        10:36:35   Log-Likelihood:                -3587.5
No. Observations:                 862   AIC:                             7187.
Df Residuals:                     856   BIC:                             7216.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -21.8908     10.949     -1.999      0.046     -43.381      -0.401
white[T.True]           4.3761      1.575      2.778      0.006       1.284       7.468
gestation               0.4446      0.039     11.510      0.000       0.369       0.520
smoke                  -8.8601      2.077     -4.267      0.000     -12.936      -4.784
smoke:white[T.True]     1.0069      2.421      0.416      0.678      -3.746       5.759
mpregwt                 0.1319      0.026      5.132      0.000       0.081       0.182
==============================================================================
Omnibus:                        8.544   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.014   Jarque-Bera (JB):               11.787
Skew:                           0.071   Prob(JB):                      0.00276
Kurtosis:                       3.555   Cond. No.                     6.35e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.35e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Exercise 6

In [312]:
hypotheses = ['smoke + smoke:white[T.True]']
t_test = model3.t_test(hypotheses)
print(t_test)

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -7.8532      1.253     -6.266      0.000     -10.313      -5.393


In [313]:
t_test.tvalue[0][0]
ex6_compare_white_black_tstat = t_test.tvalue[0][0]
ex6_compare_white_black_tstat

-6.2663856310971475

In [314]:
ex6_compare_white_black = "smaller"

# Exercise 7

In [315]:
newborns = pd.DataFrame(
    {
        "smoke": [1, 1, 0, 0],
        "white": [True, False, True, False],
        "gestation": [253, 300, 248, 287],
        "mpregwt": [132, 129, 140, 139],
    }
)
newborns



,smoke,white,gestation,mpregwt
0,1,True,253,132
1,1,False,300,129
2,0,True,248,140
3,0,False,287,139


In [316]:
pred = model3.predict(newborns)
ex7_predictions = pred.tolist()
ex7_predictions



[104.52308427743307, 119.63915539293508, 111.20881028285355, 124.0390947146368]

# Exercise 8

In [317]:
smoking.loc[smoking["gestation"] < 252, 'premature'] = 1
smoking.loc[smoking["gestation"] >= 252, 'premature'] = 0
smoking['white'] = smoking['mrace'] <6
smoking.head()


,id,date,gestation,bwt,parity,mrace,mage,med,mht,mpregwt,inc,smoke,premature,white
0,4604,1598,148,116,7,7,28,1,66,135,2,0,1.0,False
1,7435,1527,181,110,7,7,27,1,64,133,1,0,1.0,False
2,7722,1563,204,55,11,7,35,3,65,140,6,0,1.0,False
3,2026,1503,225,132,4,7,28,2,67,148,3,0,1.0,False
4,3553,1638,233,105,4,7,34,3,61,130,3,0,1.0,False


In [318]:
model4 = sm.Logit.from_formula('premature ~ smoke + mpregwt + white', data=smoking).fit()
model4.summary()

Optimization terminated successfully.
         Current function value: 0.187899
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              premature   No. Observations:                  869
Model:                          Logit   Df Residuals:                      865
Method:                           MLE   Df Model:                            3
Date:                Tue, 22 Nov 2022   Pseudo R-squ.:                 0.04614
Time:                        10:36:35   Log-Likelihood:                -163.28
converged:                       True   LL-Null:                       -171.18
Covariance Type:            nonrobust   LLR p-value:                  0.001248
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.5353      1.024     -1.499      0.134      -3.542       0.472
white[T.True]    -1.2247      0.320     -3.833      0.000      -1.851      -0.598
smoke             0.4253      0.320      1.331      0.183      -0.201       1.052
mpregwt          -0.0070      0.008     -0.907      0.364      -0.022       0.008
=================================================================================
"""

In [319]:
ex8_smoking_coefficient = model4.params['smoke']
ex8_smoking_coefficient

0.425277339783787

# Exercise 9

In [320]:
dataset= pd.read_parquet("https://github.com/nickeubank/MIDS_Data/blob/master/arrest_rates_and_population.parquet?raw=true")
dataset

variable,COUNTY,year,drug_arrest_rate,total_population,violent_arrest_rate
0,Alameda County,2009,0.003946,1457095.0,0.002963
1,Alameda County,2018,0.000652,1629615.0,0.002536
2,Alpine County,2009,0.001735,1153.0,0.006938
3,Alpine County,2018,0.000831,1203.0,0.004156
4,Amador County,2009,0.002655,38039.0,0.002629
...,...,...,...,...,...
111,Ventura County,2018,0.000863,847834.0,0.002679
112,Yolo County,2009,0.003182,192974.0,0.003031
113,Yolo County,2018,0.000564,212605.0,0.002705
114,Yuba County,2009,0.002976,70906.0,0.004993


# Exercise 10

In [321]:
average2009 = dataset[dataset["year"] == 2009]["drug_arrest_rate"].mean()
average2009

0.003191448015158338

In [322]:
dataset['year_2018'] = dataset['year'] == 2018

In [323]:
high_arrest_rate = dataset.loc[(dataset['year'] == 2009) & (dataset['drug_arrest_rate'] > average2009), 'COUNTY']
ex10_num_high_drug_arrest= len(high_arrest_rate)
ex10_num_high_drug_arrest


23

# Exercise 11

In [324]:
dataset['pop_adjusted'] = dataset['COUNTY'].map({high: 1 for high in high_arrest_rate})
dataset['pop_adjusted'] = dataset['pop_adjusted'].fillna(0)
dataset.head(5)


variable,COUNTY,year,drug_arrest_rate,total_population,violent_arrest_rate,year_2018,pop_adjusted
0,Alameda County,2009,0.003946,1457095.0,0.002963,False,1.0
1,Alameda County,2018,0.000652,1629615.0,0.002536,True,1.0
2,Alpine County,2009,0.001735,1153.0,0.006938,False,0.0
3,Alpine County,2018,0.000831,1203.0,0.004156,True,0.0
4,Amador County,2009,0.002655,38039.0,0.002629,False,0.0


In [325]:
model5 = smf.ols('violent_arrest_rate ~ C(pop_adjusted) + C(year_2018) + C(pop_adjusted)*C(year_2018)', data=dataset).fit()
model5.params

Intercept                                      0.003403
C(pop_adjusted)[T.1.0]                         0.000775
C(year_2018)[T.True]                          -0.000185
C(pop_adjusted)[T.1.0]:C(year_2018)[T.True]   -0.000247
dtype: float64

In [326]:
ex11_diff_in_diff = model5.params['C(pop_adjusted)[T.1.0]:C(year_2018)[T.True]']
ex11_diff_in_diff

-0.0002473937886135572

# Exercise 12

In [327]:
formula = 'violent_arrest_rate ~ C(pop_adjusted) + C(year_2018) + C(pop_adjusted)*C(year_2018)'
model6 = smf.wls(formula,dataset , weights=dataset['total_population']).fit()
model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             WLS Regression Results                            
===============================================================================
Dep. Variable:     violent_arrest_rate   R-squared:                       0.396
Model:                             WLS   Adj. R-squared:                  0.380
Method:                  Least Squares   F-statistic:                     24.52
Date:                 Tue, 22 Nov 2022   Prob (F-statistic):           2.86e-12
Time:                         10:36:36   Log-Likelihood:                 608.13
No. Observations:                  116   AIC:                            -1208.
Df Residuals:                      112   BIC:                            -1197.
Df Model:                            3                                         
Covariance Type:             nonrobust                                         
===============================================================================================================
                                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
Intercept                                       0.0028      0.000     21.484      0.000       0.003       0.003
C(pop_adjusted)[T.1.0]                          0.0011      0.000      5.938      0.000       0.001       0.001
C(year_2018)[T.True]                           -0.0004      0.000     -1.972      0.051      -0.001    1.76e-06
C(pop_adjusted)[T.1.0]:C(year_2018)[T.True]    -0.0003      0.000     -1.171      0.244      -0.001       0.000
==============================================================================
Omnibus:                        6.397   Durbin-Watson:                   1.073
Prob(Omnibus):                  0.041   Jarque-Bera (JB):               10.179
Skew:                           0.125   Prob(JB):                      0.00616
Kurtosis:                       4.429   Cond. No.                         7.31
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [328]:
ex12_diff_in_diff_weighted = model6.params['C(pop_adjusted)[T.1.0]:C(year_2018)[T.True]']
ex12_diff_in_diff_weighted

-0.0002883709530635655

In [329]:
results = {"ex2_birth_weight_lower": ex2_birth_weight_lower,
    "ex3_smoking_coefficient": ex3_smoking_coefficient,
    "ex4_smoking_coefficient": ex4_smoking_coefficient,
    "ex6_compare_white_black_tstat": ex6_compare_white_black_tstat,
    "ex6_compare_white_black": ex6_compare_white_black,
    "ex7_predictions": ex7_predictions,
    "ex8_smoking_coefficient": ex8_smoking_coefficient,
    "ex10_num_high_drug_arrest": ex10_num_high_drug_arrest,
    "ex11_diff_in_diff": ex11_diff_in_diff,
    "ex12_diff_in_diff_weighted": ex12_diff_in_diff_weighted,}

In [330]:
assert set(results.keys()) == {
    "ex2_birth_weight_lower",
    "ex3_smoking_coefficient",
    "ex4_smoking_coefficient",
    "ex6_compare_white_black_tstat",
    "ex6_compare_white_black",
    "ex7_predictions",
    "ex8_smoking_coefficient",
    "ex10_num_high_drug_arrest",
    "ex11_diff_in_diff",
    "ex12_diff_in_diff_weighted",
}